In [1]:
import matplotlib.pyplot as plt
import numpy as np
from   tensorflow.keras import regularizers
from   tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from   tensorflow.keras.datasets import fashion_mnist
from   tensorflow.keras.layers import Dense, Input
from   tensorflow.keras.models import Model

%matplotlib inline

In [2]:
(X_train, _), (X_test, _) = fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [3]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [4]:
# Flatten
X_train = X_train.reshape((X_train.shape[0], np.prod(X_train.shape[1:])))
X_test = X_test.reshape((X_test.shape[0], np.prod(X_test.shape[1:])))

In [5]:
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [6]:
image_dim = X_train.shape[1]
encoding_dim = 32

In [7]:
input_image = Input(shape=(image_dim,))
encoded_image = Dense(
    encoding_dim, 
    activation='relu', 
    activity_regularizer=regularizers.l1(10e-5)
)(input_image)
encoder = Model(input_image, encoded_image)
decoded_image = Dense(image_dim, activation='sigmoid')(encoded_image)
autoencoder = Model(input_image, decoded_image)

In [8]:
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [20]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [21]:
checkpointer1 = ModelCheckpoint(
    filepath='modWBest.hdf5', verbose=2, save_best_only=True)

In [22]:
checkpointer2 = EarlyStopping(monitor='val_loss', 
                              min_delta=0.0005, 
                              patience=2, 
                              verbose=2, 
                              mode='auto')

In [23]:
EPOCHS = 50
BATCH = 256
autoencoder.fit(X_train, 
                X_train, 
                epochs=EPOCHS, 
                batch_size=BATCH, 
                verbose=2, 
                shuffle=True, 
                validation_data=(X_test, X_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 - 3s - loss: 0.6750 - val_loss: 0.6748
Epoch 2/50
60000/60000 - 3s - loss: 0.6745 - val_loss: 0.6743
Epoch 3/50
60000/60000 - 3s - loss: 0.6740 - val_loss: 0.6738
Epoch 4/50
60000/60000 - 3s - loss: 0.6735 - val_loss: 0.6733
Epoch 5/50
60000/60000 - 3s - loss: 0.6730 - val_loss: 0.6728
Epoch 6/50
60000/60000 - 3s - loss: 0.6725 - val_loss: 0.6723
Epoch 7/50
60000/60000 - 3s - loss: 0.6719 - val_loss: 0.6717
Epoch 8/50
60000/60000 - 3s - loss: 0.6714 - val_loss: 0.6712
Epoch 9/50
60000/60000 - 3s - loss: 0.6708 - val_loss: 0.6706
Epoch 10/50
60000/60000 - 3s - loss: 0.6702 - val_loss: 0.6700
Epoch 11/50
60000/60000 - 3s - loss: 0.6697 - val_loss: 0.6694
Epoch 12/50
60000/60000 - 3s - loss: 0.6690 - val_loss: 0.6688
Epoch 13/50
60000/60000 - 3s - loss: 0.6684 - val_loss: 0.6682
Epoch 14/50
60000/60000 - 3s - loss: 0.6678 - val_loss: 0.6675
Epoch 15/50
60000/60000 - 3s - loss: 0.6672 - val_loss: 0.6669
Epoch 16/50
60

In [24]:
encoded_images = encoder.predict(X_test)         # compress
decoded_images = decoder.predict(encoded_images) # decompress

In [25]:
# With checkpoints and saving:
'''
autoencoder.fit(X_train,
                X_train,
                epochs=EPOCHS,
                batch_size=BATCH,
                verbose=2,
                callbacks=[checkpointer1], # or checkpointer2
                shuffle=True,
                validation_data=(X_test, X_test))
'''

'\nautoencoder.fit(X_train,\n                X_train,\n                epochs=EPOCHS,\n                batch_size=BATCH,\n                verbose=2,\n                callbacks=[checkpointer1], # or checkpointer2\n                shuffle=True,\n                validation_data=(X_test, X_test))\n'

In [26]:
#autoencoder.load_weights('modWBest.hdf5')
#encoded_images = encoder.predict(X_test)
#decoded_images = decoder.predict(encoded_images)

In [ ]:
n_items = 12
plt.figure(figsize=(20, 4))

for i in range(n_items):
    # original:
    graph = plt.subpot(2, n_items, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    graph.get_xaxis().set_visible(False)
    graph.get_yaxis().set_visible(False)
    
    # compressed/decompressed
    graph = plt.subplot(2, n_items, i + 1 + n_items)
    plt.imshow(decoded_images[i].reshape(28, 28))
    plt.gray()
    graph.get_xaxis().set_visible(False)
    graph.get_yaxis().set_visible(False)